## Assemble and visualize all downloaded S2 images

Operation for one of the following four locations

In [9]:
# Chengdu
# loc_dir = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\6468_3360_13'

# Calama
# loc_dir = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\2528_4620_13'

# Cusco
# loc_dir = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\2459_4406_13'

# Toulouse
loc_dir = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13'

Define related operation functions

In [5]:
import glob
import os
from osgeo import gdal
import numpy as np

def assemble_s2_from_bands(month_dir,out_file):
    s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    # load geoinformation from B2
    f = gdal.Open(os.path.join(month_dir,"cloud_free.B2.tif"))
    row = f.RasterYSize
    col = f.RasterXSize    
    proj = f.GetProjection()
    geoInfo = f.GetGeoTransform()
    del f
    
    # initial geotiff output
    driver = gdal.GetDriverByName('GTiff')
    f_out = driver.Create(out_file, col, row, len(s2_bands),gdal.GDT_UInt16)
    f_out.SetProjection(proj)
    f_out.SetGeoTransform(geoInfo)
    
    # load bands and assemble them into out file
    for idx_bnd in range(len(s2_bands)):
        f = gdal.Open(os.path.join(month_dir,"cloud_free." + s2_bands[idx_bnd] + ".tif"))
        dat = f.ReadAsArray()
        del f        
        outBand = f_out.GetRasterBand(idx_bnd+1)
        outBand.WriteArray(dat)
        outBand.FlushCache()    
    return 0


def s2_rgb_visualization(dat):
    mask = dat[:,:,:]>0
    mask = np.sum(mask,axis=0)>0
    col_arr = np.zeros((3,dat.shape[1],dat.shape[2]))
    for i in range(2,-1,-1):
        tmp = dat[i,:,:]
        tmp = tmp[mask]
        maxx = np.quantile(tmp,0.98)
        minn = np.quantile(tmp,0.02)
        tmp[tmp<minn]=minn
        tmp[tmp>maxx]=maxx
        tmp = (255*(tmp-minn)/(maxx-minn)).astype(np.uint8)
        col_tmp = np.zeros(dat[i,:,:].shape)
        col_tmp[mask] = tmp
        col_arr[2-i,:,:] = col_tmp
    return col_arr

def load_s2_tiff_file(path):
    f = gdal.Open(path)
    dat = f.ReadAsArray()
    row = f.RasterYSize
    col = f.RasterXSize
    bnd = f.RasterCount
    proj = f.GetProjection()
    geoInfo = f.GetGeoTransform()
    del f
    return dat, row, col, bnd, proj, geoInfo

def save_s2_rgb_tiff_visual(path, arr, col, row, bnd, proj, geoInfo):
    driver = gdal.GetDriverByName('GTiff')
    f = driver.Create(path, col, row, bnd, gdal.GDT_Byte)
    f.SetProjection(proj)
    f.SetGeoTransform(geoInfo)
    for i in range(3):
        outBand = f.GetRasterBand(i+1)
        outBand.WriteArray(arr[i])
        outBand.FlushCache()
    del outBand
    del f
    return 0


def initial_dir_if_not_exits(directory):
    if not os.path.exists(directory):
        upper_dir = '\\'.join(directory.split('\\')[:-1])
        print(upper_dir)
        if os.path.exists(upper_dir):                 
            os.mkdir(directory)
        else:
            initial_dir_if_not_exits(upper_dir)
    return 0



Main function for the operation

In [10]:
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
# get month directory
month_dir = glob.glob(os.path.join(loc_dir,'*'))

for item in month_dir:
    print(item)
    # set the directory for assemble tiff
    assemble_tif_dir = os.path.join('\\'.join(item.split('\\')[:-1]), '_'.join(item.split('\\')[-2:])+'.tif')
    assemble_tif_dir = assemble_tif_dir.replace('Sentinel-2','analysis_ready')
        
    # call assemble function
    initial_dir_if_not_exits('\\'.join(assemble_tif_dir.split('\\')[:-1]))
    assemble_s2_from_bands(item, assemble_tif_dir)
    
    # set the visualization tiff directory
    visual_tif_dir = assemble_tif_dir.replace('.tif','_visualization.tif')   
    
    # generate and save the visualization tiff file
    dat, row, col, bnd, proj, geoInfo = load_s2_tiff_file(assemble_tif_dir)
    col_arr = s2_rgb_visualization(dat)
    save_s2_rgb_tiff_visual(visual_tif_dir, col_arr, col, row, 3, proj, geoInfo)
    


C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_01
C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_02
C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_03
C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_04
C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_05
C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_06
C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_07
C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_08
C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2\4127_2991_13\2018_09
C:\Users\Jingliang\Documents\projects\Sentinel